In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
## Load data
month_data = np.load('../data/covid/covid_month.npy')
month_data

array([[3.00000000e+00, 9.00000000e+01, 1.16000000e+02, 1.24070000e+04,
        5.52000000e+02],
       [3.00000000e+00, 7.10000000e+01, 5.50000000e+01, 1.24070000e+04,
        3.72000000e+02],
       [3.00000000e+00, 2.11000000e+02, 1.35840000e+03, 1.24070000e+04,
        1.82700000e+03],
       ...,
       [8.00000000e+00, 3.01590000e+04, 2.54997750e+05, 6.44468600e+06,
        3.37580000e+04],
       [8.00000000e+00, 7.30500000e+03, 2.07651600e+05, 6.44468600e+06,
        7.98900000e+03],
       [8.00000000e+00, 4.44410000e+04, 1.69757333e+05, 6.44468600e+06,
        5.14080000e+04]])

In [11]:
month_X = month_data[:, 1:]
month_X = month_X[:,:-1]
month_y = month_data[:, -1]

In [12]:
month_X_train, month_X_test, month_y_train, month_y_test = train_test_split(
    month_X, month_y, test_size=0.4, random_state=42)

# Linear Reg

In [23]:
lin_model = LinearRegression()
lin_model.fit(month_X_train, month_y_train)

LinearRegression()

In [24]:
month_y_lin_pred = lin_model.predict(month_X_test)
lin_mse = mean_squared_error(month_y_lin_pred, month_y_test)
mse

747207503.8776608

# SVR

In [18]:
svr_model = SVR()
svr_model.fit(month_X_train, month_y_train)

SVR()

In [19]:
month_y_pred = svr_model.predict(month_X_test)

In [20]:
mse = mean_squared_error(month_y_pred, month_y_test)
mse

747207503.8776608